In [ ]:
!pip install -U albumentations transformers==4.25.1 nougat-ocr

In [ ]:
!pip install langchain langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully

In [ ]:
!nougat /content/ISLP.pdf -o /content/results/ --no-markdown --recompute --no-skipping

2024-10-16 13:46:42.467484: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-16 13:46:42.485059: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-16 13:46:42.506307: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-16 13:46:42.512731: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-16 13:46:42.528238: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
import re

with open("/content/results/ISLP.mmd","r",encoding="utf-8") as f:
  content = f.read()

In [ ]:
formulas = re.findall(r'(\$.*?\$|\$\$.*?\$\$|\\\[.*?\\\])', content)

In [ ]:
import pickle

with open("results/latex_formulas.pkl", "wb") as file:
  pickle.dump(formulas, file)

In [ ]:
import pickle

with open("results/latex_formulas.pkl", "rb") as file:
  formulas = pickle.load(file)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage
from langchain_openai.chat_models import ChatOpenAI
import os
from dotenv import load_dotenv
load_dotenv()

OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

chat = ChatOpenAI(openai_api_key=OPEN_API_KEY,model="gpt-3.5-turbo",temperature=0)

problem_example = r"\delta_k(x) = x^T \Sigma^{{-1}} \mu_k - \frac{{1}}{{2}} \mu_k^T \Sigma^{{-1}} \mu_k + \log \pi_k "

formulas_template = f"""You are an assistant who has a task to describe LaTex formula content by words.
This is example of your task: \n
Problem: {problem_example}""" + """
Solution: delta k(x) = x transpose Sigma inverse mu k - 1/2 mu k transpose Sigma inverse mu k + log pi k
You will execute your task on this formula:{formula} and return only solution.
"""
formulas_prompt = ChatPromptTemplate.from_template(formulas_template)
describe_formula_chain = formulas_prompt | chat | StrOutputParser()

formula_descriptions = [describe_formula_chain.invoke({"formula":formula}) for formula in formulas]

In [ ]:
with open("results/formula_descriptions.pkl","wb") as file:
    pickle.dump(formula_descriptions,file)

In [ ]:
with open("results/formula_descriptions.pkl","rb") as file:
    formula_descriptions = pickle.load(file)